# HW_7

In [113]:
!pip install scikit-surprise


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [114]:
import numpy as np
from surprise import Dataset, Reader, SVD, NMF, SVDpp  
from surprise.model_selection import GridSearchCV, cross_validate

In [115]:
# Створення об'єкту Reader для вказання максимального та мінімального рейтингів у датасеті
reader = Reader(rating_scale=(1, 5))

In [116]:
# Завантаження датасету Movielens
data = Dataset.load_builtin('ml-100k', prompt=False)

In [117]:
# Визначення сітки параметрів для пошуку найкращих параметрів
param_grid = {
    'n_factors': [50, 100, 150],
    'reg_pu': [0.06, 0.08, 0.1],  # регуляризація для користувачів
    'reg_qi': [0.06, 0.08, 0.1],  # регуляризація для предметів
    'n_epochs': [20, 30, 40]  # кількість епох
}

In [118]:
# Ініціалізація алгоритму SVD
algo = NMF

# Ініціалізація об'єкту GridSearchCV для крос-валідації та пошуку найкращих параметрів
grid_search = GridSearchCV(algo, param_grid, measures=['rmse'], cv=3, n_jobs=-1)

In [119]:
# Запуск пошуку найкращих параметрів
grid_search.fit(data)

In [120]:
# Виведення найкращих параметрів та результатів
print(grid_search.best_params)
print(grid_search.best_score)

{'rmse': {'n_factors': 50, 'reg_pu': 0.1, 'reg_qi': 0.1, 'n_epochs': 40}}
{'rmse': 0.9588185062792794}


In [121]:
# Створення об'єктів алгоритмів для порівняння
algos = {
    'SVD': SVD(**grid_search.best_params['rmse']),  
    'SVD++': SVDpp(**grid_search.best_params['rmse']),  
    'NMF': NMF(**grid_search.best_params['rmse'])
}

In [122]:
# Перевірка кожного алгоритму за допомогою крос-валідації та виведення результатів
for algo_name, algo_obj in algos.items():
    results = cross_validate(algo_obj, data, measures=['RMSE'], cv=3, verbose=False)  
    print(f"{algo_name}: Mean RMSE = {np.mean(results['test_rmse'])}")

SVD: Mean RMSE = 0.9276510032088923
SVD++: Mean RMSE = 0.9127741555599999
NMF: Mean RMSE = 0.9596043961818345
